## read csv

In [356]:
import pandas as pd
path = 'field_of_study_exercise.csv'

In [357]:
data = pd.read_csv(path)
data.head(10)

,level,level_name,field_of_study,academic_field
0,NaN,NaN,"-, Marketing/Marketing Management, General",marketing
1,NaN,NaN,“Wellness Counseling Certificate”,NaN
2,NaN,NaN,"(BA) Hons Fashion, Fashion/Apparel Design",NaN
3,NaN,NaN,"(BA) Liberal Arts, Liberal Arts and Sciences/L...",NaN
4,NaN,NaN,* Career certificate of Marketing.,NaN
5,NaN,NaN,"1st Class Honour, Information System and Compu...",NaN
6,NaN,NaN,"2:1, Business Management",business management
7,NaN,NaN,"2:1, French and Hispanic Studies",NaN
8,NaN,NaN,"2.1, Graphic Design and Illustration",NaN
9,NaN,NaN,"2.1, Management",NaN


## Data processing

### Remove duplicate rows:

In [358]:
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   level           322 non-null    object
 1   level_name      312 non-null    object
 2   field_of_study  375 non-null    object
 3   academic_field  141 non-null    object
dtypes: object(4)
memory usage: 11.8+ KB


### Validate Missing Values:

In [359]:
data.info()
null_counts = data.isnull()
print(" -- null count:\n{}".format(null_counts.sum()))
print(" -- null ratios:\n{}".format(null_counts.mean()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   level           322 non-null    object
 1   level_name      312 non-null    object
 2   field_of_study  375 non-null    object
 3   academic_field  141 non-null    object
dtypes: object(4)
memory usage: 11.8+ KB
 -- null count:
level              53
level_name         63
field_of_study      0
academic_field    234
dtype: int64
 -- null ratios:
level             0.141333
level_name        0.168000
field_of_study    0.000000
academic_field    0.624000
dtype: float64


When dealing with machine learning and a dataset that is both small and incomplete, filling in missing values (handling NA columns) is a necessary step to maximize the utility of the available data

### Column Cleanup:

In [360]:
# Clean column names
cleaned_data = data 
cleaned_data.columns = cleaned_data.columns.str.strip()
# Display cleaned column names
print(data.columns)

Index(['level', 'level_name', 'field_of_study', 'academic_field'], dtype='object')


### Data Type Conversion:

In [361]:
for column in cleaned_data.columns:
    cleaned_data[column]=cleaned_data[column].astype('str')

### Remove or Replace Unwanted Characters:

In [362]:
import re
# Define a regular expression to remove special characters
pattern = r'[^\w\s]'  # Matches characters that are not alphanumeric or whitespace
# Create an empty dictionary to store column-wise special characters
special_chars = set()
# Iterate through each column in the DataFrame
for col in cleaned_data.columns:
    # Create an empty set for the current column's special characters
    

    # Iterate through each element in the column
    for element in data[col].values:
        # Find all special characters using the regular expression
        matches = re.findall(pattern, str(element))
        # Add unique characters to the set
        special_chars.update(matches)

# Print the dictionary showing special characters by column
print("Special characters:")
print(f"{sorted(special_chars)}")

Special characters:
['"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', ']', '’', '“', '”']


In [363]:

mapping_specific_pattern ={
    'bba':'',
    'honours': 'Hons',
    'bachelor degree in':'',
    'bachelor degree':'',
    'bachelor in':'',
    'bachelor of':'',
    'basc': '',
    '&': 'and',
    'bachelor hons in': ''
}
# Function to replace substrings regardless of case sensitivity
def replace_substrings(string, replacements):
    pattern = re.compile('|'.join(re.escape(key) for key in replacements.keys()))
    return pattern.sub(lambda match: replacements[match.group(0).lower()], string)
# Function to clean each entry
def clean_data(field):

    # Split the field by ','
    fields = field.split(',')
    # replace special cases:
    cleaned_fields = [f.lower() for f in fields] # convert to lower case
    cleaned_fields = [replace_substrings(s, mapping_specific_pattern) for s in fields]
    # Clean each split value
    cleaned_fields = [re.sub(r'\W+', ' ', f).strip() for f in cleaned_fields] # Remove punctuation
    
    # mapping specific patterns:
    cleaned_fields = [replace_substrings(s, mapping_specific_pattern) for s in cleaned_fields]
    # Remove empty fields
    cleaned_fields = [f.lower() for f in cleaned_fields if f] # convert to lower case
    # Join them back with ','
    cleaned_text =  ', '.join(cleaned_fields)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Remove multiple spaces
    return cleaned_text
    
cleaned_data['field_of_study'] = cleaned_data['field_of_study'].apply(clean_data)
cleaned_data['academic_field'] = cleaned_data['academic_field'].apply(clean_data)
cleaned_data['level'] = cleaned_data['level'].apply(clean_data)
cleaned_data['level_name'] = cleaned_data['level_name'].apply(clean_data)
cleaned_data.replace("nan", pd.NA, inplace=True)
# cleaned_data.to_csv('cleaned_data.csv')
# data['field_of_study'] = data['field_of_study'].apply(clean_field_of_study)


In [364]:
complete_data = cleaned_data.dropna()
complete_data.to_csv('complete_data.csv',index=False)

### Validate and Standardize Values:

#### K-Nearest Neighbors (KNN) imputation

In [365]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

knn_data=cleaned_data
# Introducing a missing value in 'academic_field' column
knn_data.loc[2, 'academic_field'] = None
# Encode the 'academic_field' column
label_encoder = LabelEncoder()
knn_data['academic_field_encoded'] = label_encoder.fit_transform(knn_data['academic_field'].astype(str))
knn_data['academic_field_encoded_feature_exatraction'] = label_encoder.fit_transform(knn_data['academic_field'].astype(str))

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(','))
tfidf_matrix = tfidf_vectorizer.fit_transform(knn_data['field_of_study'])

# Prepare feature matrix with TF-IDF features
feature_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
feature_matrix['academic_field_encoded_feature_exatraction'] = knn_data['academic_field_encoded_feature_exatraction']

# Prepare the feature matrix
data_encoded = pd.get_dummies(knn_data['field_of_study'])
data_encoded['academic_field_encoded'] = knn_data['academic_field_encoded']


data_encoded = pd.concat([data_encoded, feature_matrix], axis=1)
# Impute missing values using KNNImputer
imputer = KNNImputer(n_neighbors=20) 
data_imputed = imputer.fit_transform(data_encoded)

# Decode the 'academic_field' column back to original labels
knn_data['academic_field_imputed'] = label_encoder.inverse_transform(data_imputed[:, -1].round().astype(int))

knn_data.drop(['academic_field_encoded','academic_field_encoded_feature_exatraction'], axis=1, inplace=True)
knn_data.to_csv('knn_data.csv')

/Users/hkhnhan/Documents/Code/csvprocessing/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


As a result, the data from the CSV file still does not contain enough records for some basic ML models to handle the missing data. Manual filling may be necessary.